In [3]:
import preprocessor as p  # módulo para limpiar tweets
from pymongo import MongoClient  # módulo para crear un cliente de la base de datos de mongo
import language_check  # módulo para corregir los tweets
from nltk.tokenize import TweetTokenizer  # módulo para tokenizar los tweets
import unicodedata  # módulo para tratar caracteres unicode
from nltk.corpus import stopwords  # lista de palabras basura
from nltk import FreqDist
p.set_options(p.OPT.URL)  # hace que preprocessor solo borre las URL dentro de los tweet
tknzr = TweetTokenizer()  # selecciona el tokenizador para tweets
tool = language_check.LanguageTool('es')  # trabajador de language_check para correjir

In [2]:
def strip_accents(s):
    """
    Quita los acentos para que el tokenizador de tweets no parta las palabras con tildes en 2.
    """
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def remove_stopwords(sentence, language):
    """
    Dada una frase, devuelve la frase tokenizada y sin palabras basura
    """
    return [ token for token in tknzr.tokenize(strip_accents(sentence)) if token.lower() not in stopwords.words(language) ]

def correct_word(word):
    """
    Corrije las palabras si es necesario
    """
    matches = tool.check(word)
    return language_check.correct(word, matches)

def clean_correct_tokenize(text):
    """
    Hace todo el procedimiento de limpiar, correjir, tokenizar. La lematización se hará en otro 
    programa porque requiere Python2.
    """
    clean = p.clean(text)
    matches = tool.check(text)
    corrected = language_check.correct(text, matches).lower()
    no_stop_words = remove_stopwords(corrected, 'spanish')
    tokens = [correct_word(word).lower() for word in no_stop_words]
    return tokens

In [4]:
client = MongoClient()
db = client['tweets']
col = db['training']

lens = []  # longitud en caracteres de tweets
toklens = []  # longitud en número de tokens de tweets
vocab = []  # lista que contiene todo el vocabulario

i = 0
print(col.find({}).count())
for tw in col.find():  # itera sobre los tweets guardados en la base de datos
    i += 1
    if i%1000==0:
        print(i, "tuits visitados")
    content = tw['content']  # extrae el contenido del tweet
    lens.append(len(content))  # añade la longitud en caracteres a la lista lens
    tokens = clean_correct_tokenize(content)  # crea los tokens del contenido
    col.update_one({"_id": tw["_id"]}, {'$set': {"tokens": tokens}})
    toklens.append(len(tokens))  # añade la cantidad de tokens a la lista toklens
    for tok in tokens:  # guarda los tokens de cada tweet en el vocabulario
        vocab.append(tok)
fdist = FreqDist(vocab)
vocab = list(set(vocab))  # hace que cada palabra aparezca una sola vez en el vocabulario
client.close()


68017
1000 tuits visitados
2000 tuits visitados
3000 tuits visitados
4000 tuits visitados
5000 tuits visitados
6000 tuits visitados
7000 tuits visitados
8000 tuits visitados
9000 tuits visitados
10000 tuits visitados
11000 tuits visitados
12000 tuits visitados
13000 tuits visitados
14000 tuits visitados
15000 tuits visitados
16000 tuits visitados
17000 tuits visitados
18000 tuits visitados
19000 tuits visitados
20000 tuits visitados
21000 tuits visitados
22000 tuits visitados
23000 tuits visitados
24000 tuits visitados
25000 tuits visitados
26000 tuits visitados
27000 tuits visitados
28000 tuits visitados
29000 tuits visitados
30000 tuits visitados
31000 tuits visitados
32000 tuits visitados
33000 tuits visitados
34000 tuits visitados
35000 tuits visitados
36000 tuits visitados
37000 tuits visitados
38000 tuits visitados
39000 tuits visitados
40000 tuits visitados
41000 tuits visitados
42000 tuits visitados
43000 tuits visitados
44000 tuits visitados
45000 tuits visitados
46000 tuits v

In [6]:
import pickle
import os

with open(os.path.join('pickles', 'char_tw_lens.p'), "wb") as f:
    pickle.dump(lens, f)

with open(os.path.join('pickles', 'token_tw_lens.p'), "wb") as f:
    pickle.dump(toklens, f)
    
with open(os.path.join('pickles', 'non_lema_vocab.p'), "wb") as f:
    pickle.dump(vocab, f, protocol=2)
    
with open(os.path.join('pickles', 'vocab_dist.p'), "wb") as f:
    pickle.dump(fdist, f, protocol=2)

In [5]:
client = MongoClient()
db = client['tweets']
col = db['rae']

for w in col.find({"ok":1}, {"defs":1}):
    defs = w["defs"]
    all_tok_defs = []
    for df in defs:
        tokens = clean_correct_tokenize(df)
        all_tok_defs.append(tokens)
    col.update_one({"_id": w["_id"]}, {'$set': {"tokens": all_tok_defs}})

client.close()


CursorNotFound: Cursor not found, cursor id: 25601201930